# Using Multiple Loggers

This example demonstrates the use of two different loggers and how to alter the configurations of each to help work with codes that use the {{poptus}} logging facilities.

In [1]:
# Remove input/outputs in rendering

import sys
import copy

from pathlib import Path

# We also assume that this is in the path below when loading the model function.
sys.path.append(Path.cwd())

from print_file import print_file

Our goal is to write and use a simple application that couples
* a given expensive computational model with
* a given computational method that uses the model

to find a solution to a problem of interest.  In our example, both the model and method functions use the {{poptus}} package to perform logging.

The given model was designed to accept the logging configuration as part of a larger configuration dictionary and to create the associated logger object internally for the user.  By constructing a model function that uses a logger with maximum verbosity, we see that the model doesn't log that much information.  However, what it does log is valuable output for us, so we will always configure the model to use that verbosity level.

In [2]:
import poptus

import numpy as np

from my_model import construct_model

model_cfg = {
    "Numerics": {
        "p": np.inf
    },
    "Logging": {
        "Level": poptus.LOG_LEVEL_MAX
    }
}
model = construct_model(model_cfg)
_ = model(np.zeros(2))

[Model] 
[Model] Creating model with p=inf
[Model] 
[Model] Value = 0.0 at [0. 0.]


While the method also accepts a configuration dictionary as an argument, the user is expected to provide the method with a logger object.

Since both the model and method require configuration dictionaries, we design our application to get all configuration information from a single, given JSON-format configuration file.  While we could use {{poptus}} within the application to log error messages, we prefer a minimal and simple error management scheme that just prints to standard output directly.  In any case, the method and model should already be logging their error messages appropriately.

In [3]:
import json

from my_model import construct_model
from my_method import run_method


def main(filename):
    with open(filename, "r") as fptr:
        configuration = json.load(fptr)

    model_cfg = configuration["Model"]
    method_cfg = configuration["Method"]

    try:
        model = construct_model(model_cfg)
        method_logger = poptus.create_logger(method_cfg["Logging"])
        run_method(method_cfg, model, method_logger)
    except Exception:
        print()
        print("FAILURE - Unable to apply my method to my model")
        print()
        return 1

    return 0

To start off, we use the (hopefully reasonable) starting configuration values suggested by the method developers.  To follow the method's progress, we log method output to standard output/error.  Since we're interested in reviewing all of our expensive model evaluations closely, we choose to write those to their own file.  The contents of our initial JSON file are

In [4]:
# Remove input from rendering

# There are overwritten as each step.
JSON_FILENAME = Path.cwd().joinpath("test.json")
TEST_LOG = Path.cwd().joinpath("model.log")

SPLIT_CONFIG = {
    "Model": {
        "Numerics": {
            "p": 2
        },
        "Logging": {
            "Level": poptus.LOG_LEVEL_MAX,
            "Filename": str(TEST_LOG),
            "Overwrite": True
        }
    },
    "Method": {
        "starting_point": [1.1, -2.2],
        "max_iters": 10,
        "stopping_criteria": 1.0e-5,
        "expert": {
            "factor": 0.5
        },
        "Logging": {
            "Level": poptus.LOG_LEVEL_DEFAULT
        }
    }
}

with open(JSON_FILENAME, "w") as fptr:
    json.dump(SPLIT_CONFIG, fptr, indent=4)

print_file(JSON_FILENAME)

{
    "Model": {
        "Numerics": {
            "p": 2
        },
        "Logging": {
            "Level": 4,
            "Filename": "/Users/jared/Projects/POptUS/POptUS_public/book/notebooks/model.log",
            "Overwrite": true
        }
    },
    "Method": {
        "starting_point": [
            1.1,
            -2.2
        ],
        "max_iters": 10,
        "stopping_criteria": 1e-05,
        "expert": {
            "factor": 0.5
        },
        "Logging": {
            "Level": 1
        }
    }
}


The standard output/error log messages are written to individual blocks

In [5]:
_ = main(JSON_FILENAME)

[Method] ERROR - Failed to converge within allotted budget


[POptUS] WARNING - Overwriting /Users/jared/Projects/POptUS/POptUS_public/book/notebooks/model.log
[Method] 
[Method] Starting point
[Method] 	e_1	1.1
[Method] 	e_2	-2.2
[Method] Iteration budget = 10
[Method] Stopping criteria = 1e-05
[Method] 
[Method] Iteration 1
[Method] Iteration 2
[Method] Iteration 3
[Method] Iteration 4
[Method] Iteration 5
[Method] Iteration 6
[Method] Iteration 7
[Method] Iteration 8
[Method] Iteration 9
[Method] Iteration 10

FAILURE - Unable to apply my method to my model



All model evaluations have been written to ``model.log`` and are

In [6]:
# Remove input from rendering

print_file(TEST_LOG)

[Model] 
[Model] Creating model with p=2
[Model] 
[Model] Value = 2.459674775249769 at [ 1.1 -2.2]
[Model] Value = 1.2298373876248845 at [ 0.55 -1.1 ]
[Model] Value = 0.6149186938124422 at [ 0.275 -0.55 ]
[Model] Value = 0.3074593469062211 at [ 0.1375 -0.275 ]
[Model] Value = 0.15372967345311056 at [ 0.06875 -0.1375 ]
[Model] Value = 0.07686483672655528 at [ 0.034375 -0.06875 ]
[Model] Value = 0.03843241836327764 at [ 0.0171875 -0.034375 ]
[Model] Value = 0.01921620918163882 at [ 0.00859375 -0.0171875 ]
[Model] Value = 0.00960810459081941 at [ 0.00429688 -0.00859375]
[Model] Value = 0.004804052295409705 at [ 0.00214844 -0.00429688]


While the model's log provides some insight, we can increase the method's logging level and log both model and method to the same output in the hope that seeing the combined output in chronological order might help us determine how to get convergence.

In [7]:
# Remove input from rendering

DEBUG_CONFIG = copy.deepcopy(SPLIT_CONFIG)
DEBUG_CONFIG["Model"]["Logging"] = {"Level": poptus.LOG_LEVEL_MAX}
DEBUG_CONFIG["Method"]["Logging"] = {"Level": poptus.LOG_LEVEL_MAX}

with open(JSON_FILENAME, "w") as fptr:
    json.dump(DEBUG_CONFIG, fptr, indent=4)

print_file(JSON_FILENAME)

{
    "Model": {
        "Numerics": {
            "p": 2
        },
        "Logging": {
            "Level": 4
        }
    },
    "Method": {
        "starting_point": [
            1.1,
            -2.2
        ],
        "max_iters": 10,
        "stopping_criteria": 1e-05,
        "expert": {
            "factor": 0.5
        },
        "Logging": {
            "Level": 4
        }
    }
}


In [8]:
_ = main(JSON_FILENAME)

[Method] ERROR - Failed to converge within allotted budget


[Model] 
[Model] Creating model with p=2
[Model] 
[Method] 
[Method] Starting point
[Method] 	e_1	1.1
[Method] 	e_2	-2.2
[Method] Iteration budget = 10
[Method] Stopping criteria = 1e-05
[Method] Expert configuration values
[Method] 	Factor = 0.5
[Method] 
[Method] Iteration 1
[Model] Value = 2.459674775249769 at [ 1.1 -2.2]
[Method] Scaling current point by 0.5
[Method] Iteration 2
[Model] Value = 1.2298373876248845 at [ 0.55 -1.1 ]
[Method] Scaling current point by 0.5
[Method] Iteration 3
[Model] Value = 0.6149186938124422 at [ 0.275 -0.55 ]
[Method] Scaling current point by 0.5
[Method] Iteration 4
[Model] Value = 0.3074593469062211 at [ 0.1375 -0.275 ]
[Method] Scaling current point by 0.5
[Method] Iteration 5
[Model] Value = 0.15372967345311056 at [ 0.06875 -0.1375 ]
[Method] Scaling current point by 0.5
[Method] Iteration 6
[Model] Value = 0.07686483672655528 at [ 0.034375 -0.06875 ]
[Method] Scaling current point by 0.5
[Method] Iteration 7
[Model] Value = 0.03843241836327764 a

A clear next step would be to either increase the maximum number of allowable iterations or increase the stopping criterion.  We try the latter since our evaluations can be very expensive. 

In [9]:
# Remove input from rendering

FIXED_CONFIG = copy.deepcopy(SPLIT_CONFIG)
FIXED_CONFIG["Method"]["stopping_criteria"] = 5.0e-3

with open(JSON_FILENAME, "w") as fptr:
    json.dump(FIXED_CONFIG, fptr, indent=4)

print_file(JSON_FILENAME)

{
    "Model": {
        "Numerics": {
            "p": 2
        },
        "Logging": {
            "Level": 4,
            "Filename": "/Users/jared/Projects/POptUS/POptUS_public/book/notebooks/model.log",
            "Overwrite": true
        }
    },
    "Method": {
        "starting_point": [
            1.1,
            -2.2
        ],
        "max_iters": 10,
        "stopping_criteria": 0.005,
        "expert": {
            "factor": 0.5
        },
        "Logging": {
            "Level": 1
        }
    }
}


In [10]:
_ = main(JSON_FILENAME)

[POptUS] WARNING - Overwriting /Users/jared/Projects/POptUS/POptUS_public/book/notebooks/model.log
[Method] 
[Method] Starting point
[Method] 	e_1	1.1
[Method] 	e_2	-2.2
[Method] Iteration budget = 10
[Method] Stopping criteria = 0.005
[Method] 
[Method] WARNING - Stopping criteria is suspiciously large
[Method] Iteration 1
[Method] Iteration 2
[Method] Iteration 3
[Method] Iteration 4
[Method] Iteration 5
[Method] Iteration 6
[Method] Iteration 7
[Method] Iteration 8
[Method] Iteration 9
[Method] Iteration 10
[Method] Approximated Solution = 0.004804052295409705


While we did get a solution, the warning message is unsettling.  Digging into the method's documents we determine that decreasing the expert-mode configuration value ``factor`` slightly is valid for our case.  Using the verbose debugging output, we believe that using ``factor = 0.25`` should help us achieve correct convergence with few evaluations.  We also increase the method's verbosity so that ``factor``'s value is logged.

In [11]:
# Remove input from rendering

FIXED_CONFIG = copy.deepcopy(SPLIT_CONFIG)
FIXED_CONFIG["Method"]["expert"]["factor"] = 0.25
FIXED_CONFIG["Method"]["Logging"]["Level"] = poptus.LOG_LEVEL_MIN_DEBUG

with open(JSON_FILENAME, "w") as fptr:
    json.dump(FIXED_CONFIG, fptr, indent=4)

print_file(JSON_FILENAME)

{
    "Model": {
        "Numerics": {
            "p": 2
        },
        "Logging": {
            "Level": 4,
            "Filename": "/Users/jared/Projects/POptUS/POptUS_public/book/notebooks/model.log",
            "Overwrite": true
        }
    },
    "Method": {
        "starting_point": [
            1.1,
            -2.2
        ],
        "max_iters": 10,
        "stopping_criteria": 1e-05,
        "expert": {
            "factor": 0.25
        },
        "Logging": {
            "Level": 2
        }
    }
}


In [12]:
_ = main(JSON_FILENAME)

[POptUS] WARNING - Overwriting /Users/jared/Projects/POptUS/POptUS_public/book/notebooks/model.log
[Method] 
[Method] Starting point
[Method] 	e_1	1.1
[Method] 	e_2	-2.2
[Method] Iteration budget = 10
[Method] Stopping criteria = 1e-05
[Method] Expert configuration values
[Method] 	Factor = 0.25
[Method] 
[Method] Iteration 1
[Method] Iteration 2
[Method] Iteration 3
[Method] Iteration 4
[Method] Iteration 5
[Method] Iteration 6
[Method] Iteration 7
[Method] Iteration 8
[Method] Iteration 9
[Method] Iteration 10
[Method] Approximated Solution = 9.38291463947208e-06


Finally we have a solution and interesting model evaulation values in the method's log file.

In [13]:
# Remove input from rendering

print_file(TEST_LOG)

[Model] 
[Model] Creating model with p=2
[Model] 
[Model] Value = 2.459674775249769 at [ 1.1 -2.2]
[Model] Value = 0.6149186938124422 at [ 0.275 -0.55 ]
[Model] Value = 0.15372967345311056 at [ 0.06875 -0.1375 ]
[Model] Value = 0.03843241836327764 at [ 0.0171875 -0.034375 ]
[Model] Value = 0.00960810459081941 at [ 0.00429688 -0.00859375]
[Model] Value = 0.0024020261477048525 at [ 0.00107422 -0.00214844]
[Model] Value = 0.0006005065369262131 at [ 0.00026855 -0.00053711]
[Model] Value = 0.00015012663423155328 at [ 6.71386719e-05 -1.34277344e-04]
[Model] Value = 3.753165855788832e-05 at [ 1.67846680e-05 -3.35693359e-05]
[Model] Value = 9.38291463947208e-06 at [ 4.19616699e-06 -8.39233398e-06]
